In [1]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
from math import pi, sin, cos
from datetime import datetime

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
train = pd.read_csv('Data/bike-sharing/train.csv')
test = pd.read_csv('Data/bike-sharing/test.csv')
data = pd.concat([train, test])
test_split = train.shape[0]

In [3]:
data.head()

,atemp,casual,count,datetime,holiday,humidity,registered,season,temp,weather,windspeed,workingday
0,14.395,3.0,16.0,2011-01-01 00:00:00,0,81,13.0,1,9.84,1,0.0,0
1,13.635,8.0,40.0,2011-01-01 01:00:00,0,80,32.0,1,9.02,1,0.0,0
2,13.635,5.0,32.0,2011-01-01 02:00:00,0,80,27.0,1,9.02,1,0.0,0
3,14.395,3.0,13.0,2011-01-01 03:00:00,0,75,10.0,1,9.84,1,0.0,0
4,14.395,0.0,1.0,2011-01-01 04:00:00,0,75,1.0,1,9.84,1,0.0,0


In [4]:
data['hour'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').hour, axis=1)
data['weekday'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').weekday(), axis=1)
data['month'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').month, axis=1)
data['year'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').year, axis=1)

In [5]:
data['hour_sin'] = data.apply(lambda x: sin(x['hour'] / 24.0 * 2 * pi), axis=1)
data['hour_cos'] = data.apply(lambda x: cos(x['hour'] / 24.0 * 2 * pi), axis=1)
data['weekday_sin'] = data.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
data['weekday_cos'] = data.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
data['month_sin'] = data.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
data['month_cos'] = data.apply(lambda x: cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
data['season_sin'] = data.apply(lambda x: sin(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)
data['season_cos'] = data.apply(lambda x: cos(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)

In [6]:
X_train = data[:test_split].drop(['datetime', 'casual', 'registered', 'count'], inplace=False, axis=1)
X_test = data[test_split:].drop(['datetime', 'casual', 'registered', 'count'], inplace=False, axis=1)
y_train = data['count'][:test_split]
y_test = data['count'][test_split:]

In [7]:
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
model = Sequential()
model.add(Dense(200, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

opt = Adam()
model.compile(loss='mean_squared_logarithmic_error', optimizer=opt)

In [9]:
n_epochs = 100
batch_size = 64

In [10]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
history = model.fit(X_train, y_train, shuffle=True, epochs=n_epochs, batch_size=batch_size, validation_split=0.1, verbose=1)

Train on 9797 samples, validate on 1089 samples
Epoch 1/100
9797/9797 [==============================] - 1s 87us/step - loss: 4.0808 - val_loss: 2.1124
Epoch 2/100
9797/9797 [==============================] - 0s 41us/step - loss: 0.9957 - val_loss: 1.1161
Epoch 3/100
9797/9797 [==============================] - 0s 45us/step - loss: 0.6625 - val_loss: 0.7477
Epoch 4/100
9797/9797 [==============================] - 0s 43us/step - loss: 0.5375 - val_loss: 0.5871
Epoch 5/100
9797/9797 [==============================] - 0s 44us/step - loss: 0.4466 - val_loss: 0.4750
Epoch 6/100
9797/9797 [==============================] - 0s 43us/step - loss: 0.3837 - val_loss: 0.3950
Epoch 7/100
9797/9797 [==============================] - 0s 42us/step - loss: 0.3475 - val_loss: 0.3595
Epoch 8/100
9797/9797 [==============================] - 0s 42us/step - loss: 0.3295 - val_loss: 0.3204
Epoch 9/100
9797/9797 [==============================] - 0s 43us/step - loss: 0.3138 - val_loss: 0.3109
Epoch 10/100
979

9797/9797 [==============================] - 0s 43us/step - loss: 0.0835 - val_loss: 0.1336
Epoch 79/100
9797/9797 [==============================] - 0s 43us/step - loss: 0.0825 - val_loss: 0.1311
Epoch 80/100
9797/9797 [==============================] - 0s 43us/step - loss: 0.0803 - val_loss: 0.1484
Epoch 81/100
9797/9797 [==============================] - 0s 45us/step - loss: 0.0818 - val_loss: 0.1291
Epoch 82/100
9797/9797 [==============================] - 0s 46us/step - loss: 0.0844 - val_loss: 0.1225
Epoch 83/100
9797/9797 [==============================] - 0s 45us/step - loss: 0.0790 - val_loss: 0.1378
Epoch 84/100
9797/9797 [==============================] - 0s 45us/step - loss: 0.0788 - val_loss: 0.1398
Epoch 85/100
9797/9797 [==============================] - 0s 46us/step - loss: 0.0768 - val_loss: 0.1402
Epoch 86/100
9797/9797 [==============================] - 0s 42us/step - loss: 0.0784 - val_loss: 0.1311
Epoch 87/100
9797/9797 [==============================] - 0s 45us/st